In [1]:
import os
from typing import List, Tuple
from pandas import DataFrame
import pandas as pd
import pickle
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine
import sqlite3
import asyncio
import nest_asyncio
nest_asyncio.apply()

from src.utils import (
    pdf_data_loader,
    get_nodes,
concat_node_object,
    get_index,
    get_sentence_window_query_engine,
load_data_to_sql_db,
csv_excel_data_loader,
text_to_query_engine
    )

from llama_index.core import Document

load_dotenv(find_dotenv())

True

In [4]:
cur = Path.cwd()
cur

PosixPath('/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/src')

In [6]:
cur.parent / "data" /"pdfs"

PosixPath('/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/pdfs')

In [7]:
# File Paths
current_path = Path.cwd()
pdf_dirs = current_path.parent / "data" / "pdfs"
# pdf_dirs  = Path("/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/pdfs")
docs_path = current_path.parent / "data" / "llamadocs"
# docs_path = Path("/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/llamadocs")
chroma_path = current_path.parent / "data" / "chromadb"
# chroma_path = Path("/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/chromadb")
sqlitepath = current_path.parent / "data" / "sqlite" / "sqlite.db"
# sqlitepath = Path("/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/sqlite/sqlite.db")
# Tesla Data
nodes_path = current_path.parent / "data" / "nodes" / "tesla_esg_nodes.pkl"
# nodes_path = Path("/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/nodes/tesla_esg_nodes.pkl")


# Sales Data
sales_data_path = current_path.parent / "data" / "csv" / "kaggle_sample_superstore.csv"
# sales_data_path = Path("/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/csv/kaggle_sample_superstore.csv")
sales_nodes_path = current_path.parent / "data" / "nodes" / "sales_nodes.pkl"
# sales_nodes_path = Path("/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/nodes/sales_nodes.pkl")


# Process data to generate Document object

In [13]:
def create_doc(filepath: Path,
               docs_dir: Path = None,
               docs_file_name: str = None,
               db_path: Path = None,
               tablename: str = None,
               pdf: bool = True,
               embed_csv: bool = True,
               embed_columns: list = None,
               metadata_column: list = None,
               load_sql: bool = False,
               ) -> Tuple[list[Document], DataFrame]:
    docs = None
    df = DataFrame()
    pdf_file_path = []

    if docs_dir is not None and docs_file_name is not None:
        save_docs_file_path = Path(os.path.join(docs_dir, docs_file_name))
    else:
        save_docs_file_path = None

    if save_docs_file_path is not None and save_docs_file_path.exists():
        with save_docs_file_path.open("rb") as file:
            docs = pickle.load(file)
    else:
        if pdf:
            docs = []
            # for filename in os.listdir(filepath):
            #     file_path =  Path(os.path.join(filepath, filename))
                # pdf_file_path.append(file_path)
            docs = asyncio.run(pdf_data_loader(filepath, num_workers=4))
                # docs.extend(newdoc)
            if save_docs_file_path is not None:
                with save_docs_file_path.open("wb") as file:
                    pickle.dump(docs, file)
        elif embed_csv:
            docs, _ = csv_excel_data_loader(filepath,
                                            embed_columns,
                                            metadata_column)
            if save_docs_file_path is not None:
                with save_docs_file_path.open("wb") as file:
                    pickle.dump(docs, file)
    
    if load_sql and db_path is not None and tablename is not None:
        df = load_data_to_sql_db(str(filepath), str(db_path), tablename)

    return docs, df

In [14]:
# Get Nodes
def get_query_engine(nodes_path: Path = Path(),
                    docs: List[Document] = None, 
                    chroma_path: Path = None,
                    index_name: str = None,
                    window: bool = False,
                    markdown: bool = False,
                    db_path: Path = None,
                    tablename: str = None,
                    load_sql: bool = False,):
    
    if load_sql:
        # Create a connection to the SQLite database and an engine
        # conn = sqlite3.connect(str(db_path))
        engine = create_engine("sqlite:///" + str(db_path))
        query_engine = text_to_query_engine([tablename], engine)
    elif nodes_path.exists() and not load_sql:
        with nodes_path.open("rb") as file:
            nodes = pickle.load(file)
        chroma_path = str(chroma_path)
        index = get_index(chroma_path, index_name, nodes = nodes)
        query_engine = get_sentence_window_query_engine(index, window=window)
    elif not nodes_path.exists() and not load_sql:
        nodes_minus_object, nodes_object = get_nodes(docs, is_markdown=markdown, node_save_path=nodes_path)
        nodes = concat_node_object(nodes_minus_object, nodes_object)
        with nodes_path.open("wb") as file:
            pickle.dump(nodes, file)
        chroma_path = str(chroma_path)
        index = get_index(chroma_path, index_name, nodes = nodes)
        query_engine = get_sentence_window_query_engine(index, window=window)
   
        
    return query_engine

In [15]:
esgpath = os.path.join(docs_path, "10K_esg")
with open(esgpath, "rb") as file:
    esg_docs = pickle.load(file)

esg_docs, _ = create_doc(pdf_dirs, 
                        docs_path, 
                        "10K_esg", 
                        pdf=True, 
                        embed_csv=False)


esg_query_engine = get_query_engine(nodes_path, 
                                    esg_docs, 
                                    chroma_path, 
                                    "tesla_esg", 
                                    window=False, 
                                    markdown=True)

loading index tesla_esg


In [20]:
# esg_query ="What are the key takeaways from Tesla's 2022 ESG report?"
# esg_query ="Compare Deloitte's sustainbaility goals with that of McKinsey" 
esg_query = "How much was Tesla's Selling General and Administrative expense in 2022 as a percentage of revenue?"
# esg_query = "How many McKinsey employee spent their time on sustainability work in 2022?"
response = esg_query_engine.query(esg_query)
print(response)

Tesla's Selling, General and Administrative (SG&A) expense in 2022 was 5% of revenue.


In [21]:
_, df = create_doc(filepath=sales_data_path,
                   db_path= sqlitepath,
                   tablename = "Sales",
                   pdf = False,
                   load_sql=True,
                   embed_csv=False,)
# df = load_data_to_sql_db(sales_data_path, str(sqlitepath), "Sales")
sales_sql_query_engine = get_query_engine(
                                         db_path=sqlitepath, 
                                         tablename="Sales", 
                                         load_sql=True)

In [22]:
# sales_query ="What is the total sales in Los Angeles"
sales_query ="Which Segment sold the most?" 
response = sales_sql_query_engine.query(sales_query)
print(response)

Based on the SQL query provided, the segment that sold the most is the "Consumer" segment, with a total sales of 1,161,401.34.

The SQL query first groups the sales data by the "Segment" column, calculates the total sales for each segment using the SUM(Sales) function, orders the results by the total sales in descending order, and then takes the first (LIMIT 1) result, which represents the segment with the highest total sales.

Therefore, the response to the given query "Which Segment sold the most?" would be:

"The Consumer segment sold the most, with a total sales of 1,161,401.34."


In [55]:
response.metadata

{'9ec5355c-99ad-4eef-8fe4-de335d8b7ba9': {},
 'sql_query': 'SELECT Segment, SUM(Quantity) AS TotalQuantity\nFROM Sales\nGROUP BY Segment\nORDER BY TotalQuantity DESC\nLIMIT 1;',
 'result': [('Consumer', 19521)],
 'col_keys': ['Segment', 'TotalQuantity']}